In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.optimize import minimize, Bounds
import plotly.express as px
import math
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from tqdm.auto import tqdm
# импортируем генераторы процессов с разными рычагами и KL метрику
from leverage_functions_and_metrics import gen_e, gen_gjr, gen_poly, gen_t, gen_lin,gen_multy_poly, multy_poly_leverage  
from leverage_functions_and_metrics import l_e, l_gjr, l_poly, l_t, KL, expectation_2m_
import time
import warnings
# warnings.filterwarnings('ignore')

__Вспомогательные функции для совместного "правдоподобия"__

In [42]:
def vol_egarch(x_e, eps):
    n = len(eps)
    nu_e = np.array([np.nan]*n)
    logsigma_e2 = np.array([np.nan]*n)
    logsigma_e2[0] = np.log(x_e[-1]**2)
    w_e = x_e[0]
    a_e = x_e[1]
    b_e = x_e[2]
    theta_e = x_e[3]
    nu_e[0] = eps[0]/np.sqrt(np.exp(logsigma_e2[0]))
    for t in range(1,n):
        logsigma_e2[t] = w_e + a_e*(theta_e*nu_e[t-1] + ( abs(nu_e[t-1])-np.sqrt(2/np.pi) )) + b_e*logsigma_e2[t-1]
        nu_e[t] = eps[t]/np.sqrt(np.exp(logsigma_e2[t]))
    return np.exp(logsigma_e2)
def vol_poly(x_poly, eps):
    n = len(eps)
    nu_p = np.array([np.nan]*n)
    logsigma_poly2 = np.array([np.nan]*n)
    logsigma_poly2[0] = np.log(x_poly[-1]**2)
    w_p = x_poly[0]
    a_p = x_poly[1]
    b_p = x_poly[2]
    theta_p1 = x_poly[3]
    theta_p2 = x_poly[4]
    nu_p[0] = eps[0]/np.sqrt(np.exp(logsigma_poly2[0]))
    for t in range(1,n):
        logsigma_poly2[t] = w_p + theta_p1*nu_p[t-1] + theta_p2*(nu_p[t-1]**2-1) + b_p*logsigma_poly2[t-1]
        nu_p[t] = eps[t]/np.sqrt(np.exp(logsigma_poly2[t]))
    return np.exp(logsigma_poly2)
def vol_gjr(x_gjr, eps):
    n =len(eps)
    nu_gjr = np.array([np.nan]*n)
    logsigma_gjr2 = np.array([np.nan]*n)
    logsigma_gjr2[0] = np.log(x_gjr[-1]**2)
    w_gjr = x_gjr[0]
    a_gjr = x_gjr[1]
    b_gjr = x_gjr[2]
    theta_gjr = x_gjr[3]
    nu_gjr[0] = eps[0]/np.sqrt(np.exp(logsigma_gjr2[0]))
    for t in range(1,n):
        logsigma_gjr2[t] = w_gjr + (a_gjr +theta_gjr * (eps[t-1]<0))*eps[t-1]**2 + b_gjr*logsigma_gjr2[t-1]
        nu_gjr[t] = eps[t]/np.sqrt(np.exp(logsigma_gjr2[t]))
    return np.exp(logsigma_gjr2)
def vol_tgarch(x_t, eps):
    n = len(eps)
    nu_t = np.array([np.nan]*n)
    logsigma_t2 = np.array([np.nan]*n)
    logsigma_t2[0] = np.log(x_t[-1]**2)
    w_t = x_t[0]
    b_t = x_t[1]
    theta_plus_t = x_t[2]
    theta_minus_t = x_t[3]
    nu_t[0] = eps[0]/np.sqrt(np.exp(logsigma_t2[0]))
    for t in range(1,n):
        logsigma_t2[t] = w_t + theta_plus_t*eps[t-1]*(eps[t-1]>=0) - theta_minus_t*eps[t-1]*(eps[t-1]<0) + b_t*logsigma_t2[t-1]
        nu_t[t] = eps[t]/np.sqrt(np.exp(logsigma_t2[t]))
    return np.exp(logsigma_t2)

__Запишем "правдоподобие" CMAECV для моделей EGARCH, EGARCH с полиномиальным рычагом, GJR-GARCH и TGARCH__

In [127]:
def L(x, eps, lambda_):
    x_e, x_poly, x_gjr, x_t, w = x[:5], x[5:11], x[11:16], x[16:21], np.array(x[21:])
    sigma2 = np.array([vol_egarch(x_e,eps), vol_poly(x_poly,eps),
                                                  vol_gjr(x_gjr,eps), vol_tgarch(x_t, eps)])
    h = w@sigma2
    return 0.5*np.sum(np.log(h)) + np.sum(eps/(2*h)) + lambda_*np.sum(np.abs(w*w)) #+n*np.log(2*np.pi)*0.5

__Тест на симуляции__

In [131]:
np.random.seed(42)
n = 10000
nu = np.random.normal(0,1, n)
x_lin = np.array([0.118, 0.24, 0.319, 0.286, -0.25, 0.441, -0.24, 0.32, -0.28 ,0.13, -0.145, 0.15, 0.58, 0.9])
sigma_lin = gen_lin(x_lin, nu)
eps = sigma_lin*nu

In [132]:
x = [0.2,0.3,0.1, -0.5, 0.9,
     0.2,0.3,0.1, -0.5,0.15, 0.9,
     0.02,0.03,0.01, 0.05, 0.9,
     0.2,0.3,0.1, -0.5, 0.9,
     0.25, 0.25,0.25,0.25]

In [133]:
L(x, eps, lambda_=1)

1036.6865465765743